# Combine data and Cleaning
This notebook:
* combines all the exogenous data
* removes exogenous variables that start too late or end too early
* performs grangers causality test to then remove non-useful exogenous variables

## Imports

In [ ]:
import pandas as pd
from datetime import datetime
from pathlib import Path
pd.set_option('display.max_columns', None)

In [ ]:
workbookdir = Path.cwd() #path to current folder
 
datasetsPath= workbookdir.parent / 'Data' /'Raw' #Path to data

## Declare start and end dates
**Change these values to alter the dataset**


In [ ]:
#could import helper and use as suggested below
#from data_manipulation_helpers import getTestDate
START_YEAR=1990
END_YEAR=2024
TEST_DATE= datetime(2024,1,1).strftime('%m/%Y')
#usage example (first argument is the test set size ratio)
#TEST_DATE = getTestDate(0.1, START_YEAR, END_YEAR)

TARGET='PCEPI'

These values are used to detect discontinued data or data which starts after the starting period.
If an exogenous vatiable starts 6 months late or ends 6 months early, it will be **dropped**.

In [ ]:
startDate= datetime(START_YEAR,1,1).strftime('%m/%Y')
startDate_plus_half_year=datetime(START_YEAR,6,1).strftime('%m/%Y')
endDate= datetime(END_YEAR,12,1,1).strftime('%m/%Y')
endDate_minus_half_year=datetime(END_YEAR,7,1).strftime('%m/%Y')

# Combine Data:
### Useful functions:

In [ ]:
from data_manipulation_helpers import str_to_float, rename_cols, remove_percent, interpolateAndRoundColumn, getMissingCols, printMissingCols, is_granger_caused

## Import fred data
this data includes the target variable (PCEPI **and its varients**) and has been combined from fred: https://fred.stlouisfed.org/categories

Fred kindly allows data to be combined into a data list, which can then be downloaded (instead of indivigually downloading each dataset and then combining)

In [ ]:
# Monthly data has been split into 3 CSV files and needs to be combined:

#read dataframes:
df1= pd.read_csv(datasetsPath/'monthly_1.csv', parse_dates=[0])
df2= pd.read_csv(datasetsPath/'monthly_2.csv', parse_dates=[0])
df3= pd.read_csv(datasetsPath/'monthly,_end_of_period.csv', parse_dates=[0])

#Make a date-time index
df1 = df1.set_index(['observation_date'],drop = True)
df2 = df2.set_index(['observation_date'],drop = True)
df3 = df3.set_index(['observation_date'],drop = True)

#Combine Monthly data:
combinedDf= pd.concat([df1,df2,df3],axis=1,join='outer')


#Rename and display combined data:
rename_cols(combinedDf,'fred')
display(combinedDf)


fred_TERMCBAUTO48NS and fred_TERMCBPER24NS are not actually monthly, but mid-quarterly. Linearly interpolate between observation dates (as is performed in FRED visualisation) to give 01/01/1990 date.

In [ ]:
combinedDf = interpolateAndRoundColumn(combinedDf, 'fred_TERMCBAUTO48NS', method='linear')
combinedDf = interpolateAndRoundColumn(combinedDf, 'fred_TERMCBPER24NS', method='linear')

#### Fred quarterly data

In [ ]:
# Combine quarterly data:

fred_quartely_1= pd.read_csv(datasetsPath/'quarterly.csv', parse_dates=[0])
fred_quartely_2= pd.read_csv(datasetsPath/'quarterly,_end_of_quarter.csv', parse_dates=[0])
fred_quartely_3= pd.read_csv(datasetsPath/'quarterly,_end_of_period.csv', parse_dates=[0])

# Set Date-time index
fred_quartely_1 = fred_quartely_1.set_index(['observation_date'],drop = True)
fred_quartely_2 = fred_quartely_2.set_index(['observation_date'],drop = True)
fred_quartely_3 = fred_quartely_3.set_index(['observation_date'],drop = True)

#Combine data:
quarterly=pd.concat([fred_quartely_1,fred_quartely_2,fred_quartely_3],axis=1,join='outer')


# Remove data which starts too late or ends to early (used to find the quarterly feature names so they font get removed for having too mant null values (nature of converting quarterly to monthly)):
late_data=quarterly.loc[startDate:startDate_plus_half_year].isna().all().loc[quarterly.loc[startDate:startDate_plus_half_year].isna().all()==True].index
quarterly.drop(late_data,axis=1,inplace=True)
early_data=quarterly.loc[endDate_minus_half_year:endDate].isna().all().loc[quarterly.loc[endDate_minus_half_year:endDate].isna().all()==True].index
quarterly.drop(early_data,axis=1,inplace=True)

#Rename:
rename_cols(quarterly,'fred')

quarterlyCols=quarterly.columns # Save the column names

#Combine with monthly data:
combinedDf= pd.concat([combinedDf,quarterly],axis=1,join='outer')
combinedDf= combinedDf.loc[startDate:endDate]
display(combinedDf)

### Change the index to month/year

In [ ]:
combinedDf.set_index(pd.to_datetime(pd.Series(combinedDf.index)).dt.strftime('%m/%Y'),drop = True,inplace=True)
combinedDf

#### Fred daily and weekly data
**aggregates data for each month into one value**
Achieves this by removing data which starts too late or ends too early (same as later), then takes the arithmetic mean for that month.

**NOTE:** arithmetic mean should not be used for variables which involve change/percentage change/rates of change (geometric or harmonic means must be used instead). However for the sake of simplicity, all variables involving changes have been dropped excluding their levels.

In [ ]:
# Read daily data
dailyDf=pd.read_csv(datasetsPath/'daily.csv', parse_dates=[0])

# Change to date-time index
dailyDf = dailyDf.set_index(['observation_date'],drop = True)


# Remove data that starts too late or ends too early:
late_data=dailyDf.loc[startDate:startDate_plus_half_year].isna().all().loc[dailyDf.loc[startDate:startDate_plus_half_year].isna().all()==True].index
dailyDf.drop(late_data,axis=1,inplace=True)
early_data=dailyDf.loc[endDate_minus_half_year:endDate].isna().all().loc[dailyDf.loc[endDate_minus_half_year:endDate].isna().all()==True].index
dailyDf.drop(early_data,axis=1,inplace=True)

# Select the data with the dates of interest
dailyDf= dailyDf.loc[startDate:endDate]

# Remove features involving change:
dropCols= []

for i in dailyDf.columns:
    if '_' in i:
        dropCols.append(i)

dailyDf.drop(dropCols,axis=1,inplace=True)

# Take the mean of the month:
dailyDf=dailyDf.resample('M').mean()

# Rename columns
rename_cols(dailyDf,'fred')

display(dailyDf)
dailyDf.set_index(pd.to_datetime(pd.Series(dailyDf.index)).dt.strftime('%m/%Y'),drop = True,inplace=True)

# Combine data
combinedDf=pd.merge(combinedDf,dailyDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)


In [ ]:
#read weekly data:
weeklyDf=pd.read_csv(datasetsPath/'weekly,_ending_thursday.csv', parse_dates=[0])
weeklyDf = weeklyDf.set_index(['observation_date'],drop = True)

# Remove data that starts too late or ends too early:
late_data=weeklyDf.loc[startDate:startDate_plus_half_year].isna().all().loc[weeklyDf.loc[startDate:startDate_plus_half_year].isna().all()==True].index
weeklyDf.drop(late_data,axis=1,inplace=True)
early_data=weeklyDf.loc[endDate_minus_half_year:endDate].isna().all().loc[weeklyDf.loc[endDate_minus_half_year:endDate].isna().all()==True].index
weeklyDf.drop(early_data,axis=1,inplace=True)

# Select the data with the dates of interest
weeklyDf= weeklyDf.loc[startDate:endDate]

# Remove features involving change:
dropCols= []

for i in weeklyDf.columns:
    if '_' in i:
        dropCols.append(i)

weeklyDf.drop(dropCols,axis=1,inplace=True)

# Take the mean of the month:
weeklyDf=weeklyDf.resample('M').mean()

# rename columns
rename_cols(weeklyDf,'fred')

display(weeklyDf)
weeklyDf.set_index(pd.to_datetime(pd.Series(weeklyDf.index)).dt.strftime('%m/%Y'),drop = True,inplace=True)

# combine:
combinedDf=pd.merge(combinedDf,weeklyDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

## Other exogenous Data:

### Crude oil:
https://www.investing.com/commodities/crude-oil

In [ ]:
name='CrudeOilWTI'

#read data:
tempDf= pd.read_csv(datasetsPath/f'{name}.csv', parse_dates=[0])

tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y') # convert to just month and year
#set date as index
tempDf.set_index(['Date'],drop = True,inplace=True)

#remove the order of magnitude symbols:
tempDf['Vol.']=tempDf['Vol.'].apply(str_to_float)

# Remove percent sign:
tempDf['Change %']=tempDf['Change %'].apply(remove_percent)

rename_cols(tempDf,name)

display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)


### Brent oil:
https://www.investing.com/commodities/brent-oil

In [ ]:
name='BrentOil'

tempDf= pd.read_csv(datasetsPath/f'{name}.csv', parse_dates=[0])

tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')

tempDf.set_index(['Date'],drop = True,inplace=True)

#remove the order of magnitude symbols:
tempDf['Vol.']=tempDf['Vol.'].apply(str_to_float)

# Remove percent sign:
tempDf['Change %']=tempDf['Change %'].apply(remove_percent)
rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)


### Natural Gas (Henry Hub Natural Gas Spot Price Dollars per Million Btu):
https://www.eia.gov/dnav/ng/hist/rngwhhdm.htm

(replaced original investing.com offer of futures (https://www.investing.com/commodities/natural-gas), since this had missing data from the first 3 months)

**Removed as could not find a dataset containing the first 4 months of 1990.**

### Gold:
https://www.investing.com/commodities/gold

In [ ]:
name='Gold'

tempDf= pd.read_csv(datasetsPath/f'{name}.csv', parse_dates=[0])
tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')

tempDf.set_index(['Date'],drop = True,inplace=True)
#remove the order of magnitude symbols:
tempDf['Vol.']=tempDf['Vol.'].apply(str_to_float)

# Remove percent sign:
tempDf['Change %']=tempDf['Change %'].apply(remove_percent)

rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### Silver:
https://www.investing.com/commodities/silver

In [ ]:
name='Silver'

tempDf= pd.read_csv(datasetsPath/f'{name}.csv', parse_dates=[0])
tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')

tempDf.set_index(['Date'],drop = True,inplace=True)

#remove the order of magnitude symbols:
tempDf['Vol.']=tempDf['Vol.'].apply(str_to_float)

# Remove percent sign:
tempDf['Change %']=tempDf['Change %'].apply(remove_percent)

rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### Copper:
https://www.investing.com/commodities/copper

In [ ]:
name='Copper'

tempDf= pd.read_csv(datasetsPath/f'{name}.csv', parse_dates=[0])
tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')

tempDf.set_index(['Date'],drop = True,inplace=True)

#remove the order of magnitude symbols:
tempDf['Vol.']=tempDf['Vol.'].apply(str_to_float)

# Remove percent sign:
tempDf['Change %']=tempDf['Change %'].apply(remove_percent)

rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### US Soybean:
Already included via the FRED-derived PSOYBUSDM variable (which has better coverage from 1990-).

### Baltic Dry Index:
https://uk.investing.com/indices/baltic-dry

In [ ]:
name='BalticDryIndex'

tempDf= pd.read_csv(datasetsPath/f'{name}.csv', parse_dates=[0],dayfirst=True)
tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')

tempDf.set_index(['Date'],drop = True,inplace=True)


tempDf.drop('Vol.',axis=1,inplace=True)# vol. not used

# Remove percent sign:
tempDf['Change %']=tempDf['Change %'].apply(remove_percent)

rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### Nasdaq Composite
https://www.investing.com/indices/nasdaq-composite

In [ ]:

name='NASDAQComposite'

tempDf= pd.read_csv(datasetsPath/f'{name}.csv', parse_dates=[0],dayfirst=True)
tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')
tempDf.set_index(['Date'],drop = True,inplace=True)

tempDf.drop('Vol.',axis=1,inplace=True)# vol. not used

# Remove percent sign:
tempDf['Change %']=tempDf['Change %'].apply(remove_percent)

rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### S&P 500
https://www.investing.com/indices/us-spx-500-historical-data

In [ ]:
name='SP500'

tempDf= pd.read_csv(datasetsPath/f'{name}.csv', parse_dates=[0])
tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')
tempDf.set_index(['Date'],drop = True,inplace=True)

tempDf.drop('Vol.',axis=1,inplace=True) # vol. not used
# Remove percent sign:
tempDf['Change %']=tempDf['Change %'].apply(remove_percent)

rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### CBOE VIX
Chicago Board Options Exchange Volatility Index
https://www.investing.com/indices/volatility-s-p-500-historical-data

Augmented with 01/01/1990 data from
https://finance.yahoo.com/quote/%5EVIX/history/?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbS8&guce_referrer_sig=AQAAANwY8S-Sgd81YqLYXYfh9HiLXqf649edKr-K5ZwFLKKG9jKBZwoE1VPX7-CHVWmn2KtIsjlk8mRQmccZsAXuwrw1jzb1LO-owPdHemT5Y5tsgIbcB9T4JxwkPlOORz4svx8Vj8wNyGIatWqcKw5iWTnpnH4Szw6VynT1gnTn3rHf&frequency=1mo&period1=631267200&period2=1741608394

This matches the main dataset, but includes earlier dates from which the first datapoint needed (January 1990) can be derived.

In [ ]:
name='CBOEVIX'

tempDf= pd.read_csv(datasetsPath/f'{name}.csv', parse_dates=[0])
display(tempDf['Date'])
tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')
tempDf.set_index(['Date'],drop = True,inplace=True)

tempDf.drop('Vol.',axis=1,inplace=True) # vol. not used
# Remove percent sign:
tempDf['Change %']=tempDf['Change %'].apply(remove_percent)

rename_cols(tempDf,name)
#display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
#display(combinedDf)

### food price indices data

https://www.fao.org/worldfoodsituation/foodpricesindex/en/

In [ ]:
name='food_price_indices_data_f'

tempDf= pd.read_csv(datasetsPath/f'{name}.csv', parse_dates=[0],header=2,usecols=list(range(0,7)))
tempDf.drop(0,axis=0,inplace=True)

tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')
tempDf.set_index(['Date'],drop = True,inplace=True)

rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### BCI
https://www.oecd.org/en/data/indicators/business-confidence-index-bci.html?oecdcontrol-b2a0dbca4d-var3=1974-06&oecdcontrol-b2a0dbca4d-var4=2024-12

In [ ]:
name='BCI'

tempDf= pd.read_csv(datasetsPath/f'{name}.csv', parse_dates=[0])
tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')
tempDf.set_index(['Date'],drop = True,inplace=True)

tempDf.sort_index(inplace=True)

rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### gscpi
https://www.newyorkfed.org/research/policy/gscpi#/interactive

In [ ]:
name='gscpi_data'

tempDf= pd.read_excel(datasetsPath/f'{name}.xls',sheet_name='GSCPI_Monthly_Data',engine=None,skiprows=4,names=['Date','gscpi'],usecols=[0,1])

tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')
tempDf.set_index(['Date'],drop = True,inplace=True)

rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### UK policy uncertainty Index
https://www.policyuncertainty.com/

In [ ]:
name='UK_Policy_Uncertainty_Data'

tempDf= pd.read_excel(datasetsPath/f'{name}.xlsx',engine=None,skipfooter=1)

tempDf['Date']=tempDf['year'].astype(str)+'/'+tempDf['month'].astype(str)
tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')

tempDf.drop(['year','month'],axis=1,inplace=True)

tempDf.set_index(['Date'],drop = True,inplace=True)

rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### US policy uncertainty Index
https://www.policyuncertainty.com/

In [ ]:
name='US_Policy_Uncertainty_Data'

tempDf= pd.read_excel(datasetsPath/f'{name}.xlsx',sheet_name='Main Index',engine=None,skipfooter=1)

#Combine year and month columns to form a date:
tempDf['Date']=tempDf['Year'].astype(str)+'/'+tempDf['Month'].astype(str)
tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')

tempDf.drop(['Year','Month'],axis=1,inplace=True)

tempDf.set_index(['Date'],drop = True,inplace=True)


rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

In [ ]:
name='US_Policy_Uncertainty_Data'

tempDf= pd.read_excel(datasetsPath/f'{name}.xlsx',sheet_name='Components',engine=None,skipfooter=1)

#Combine year and month columns to form a date:
tempDf['Date']=tempDf['Year'].astype(str)+'/'+tempDf['Month'].astype(str)
tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')

tempDf.drop(['Year','Month'],axis=1,inplace=True)

tempDf.set_index(['Date'],drop = True,inplace=True)


rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### World Bank Commodity Price Data
https://thedocs.worldbank.org/en/doc/5d903e848db1d1b83e0ec8f744e55570-0350012021/related/CMO-Historical-Data-Monthly.xlsx

In [ ]:
name='CMO-Historical-Data-Monthly'

tempDf= pd.read_excel(datasetsPath/f'{name}.xlsx',sheet_name='Monthly Prices',engine=None,skiprows=4)
tempDf.drop(0,axis=0,inplace=True)

#Replace M with '/' to turn it into a valid date:
tempDf.iloc[:,0]=tempDf.iloc[:,0].str.replace('M','/')
tempDf.iloc[:,0]=pd.to_datetime(tempDf.iloc[:,0]).dt.strftime('%m/%Y')

tempDf.set_index(tempDf.columns[0],drop = True,inplace=True)

#Drop already used features (and ['Barley','Sorghum','Wheat, US SRW'] because they contain '…' with different encodings):
tempDf.drop(['Crude oil, average','Crude oil, Brent','Crude oil, Dubai','Crude oil, WTI', 'Natural gas, US','Natural gas, Europe','Liquefied natural gas, Japan',
             'Natural gas index','Soybeans','Copper','Gold','Silver','Barley','Sorghum','Wheat, US SRW'],axis=1,inplace=True)

# turn '…' into NaN
tempDf.replace('…',pd.NA,inplace=True)

rename_cols(tempDf,name)
display(tempDf.iloc[-1,-1])

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

In [ ]:
name='CMO-Historical-Data-Monthly'

tempDf= pd.read_excel(datasetsPath/f'{name}.xlsx',sheet_name='Monthly Indices',engine=None,skiprows=8, names=['Date','Total_Index','Energy','Non_Energy','Agriculture',
                                                                                                              'Beverages','Food','Oils_and_meals','Grains','Other_foods','Raw_materials',
                                                                                                              'Timber','Other_raw_materials','Fertilizers','Metals_minerals','Base metals', 'Precious_metals'])
#replace 'M' with slash to turn it into a date:
tempDf['Date']=tempDf['Date'].str.replace('M','/')

tempDf['Date']=pd.to_datetime(tempDf['Date']).dt.strftime('%m/%Y')
tempDf.set_index(tempDf.columns[0],drop = True,inplace=True)


rename_cols(tempDf,name)
display(tempDf)

combinedDf=pd.merge(combinedDf,tempDf,how= 'left',left_index=True,right_index=True)
display(combinedDf)

### FTSE 100
https://uk.finance.yahoo.com/quote/%5EFTSE/

(**Data contains errorous dates hence why it has been excluded**)


## Check for duplicate values:

In [ ]:
combinedDf.index.duplicated().any()

# Remove late/early data:
Remove data that misses the first 6 months of `START_YEAR` **or** the last 6 months of `END_YEAR`

In [ ]:
late_data=combinedDf.loc[startDate:startDate_plus_half_year].isna().all().loc[combinedDf.loc[startDate:startDate_plus_half_year].isna().all()==True].index
for i in late_data:
    print(i)

combinedDf.drop(list(late_data),axis=1,inplace=True)

In [ ]:
early_data=combinedDf.loc[endDate_minus_half_year:endDate].isna().all().loc[combinedDf.loc[endDate_minus_half_year:endDate].isna().all()==True].index
#for i in early_data:
#    print(i)

combinedDf.drop(list(early_data),axis=1,inplace=True)

In [ ]:
#combinedDf

In [ ]:
print(len(combinedDf.columns))

In [ ]:
null_cols=combinedDf.drop(quarterlyCols,axis=1).isna().sum(axis=0).loc[combinedDf.drop(quarterlyCols,axis=1).isna().sum(axis=0)>combinedDf.shape[0]//5]

#display(null_cols)

Upon further inspection for TB1YR, there is a big gap between 2001 and 2008. hence all data relating to this variable should be removed (https://fred.stlouisfed.org/series/TB1YR).

For TERMCBAUTO48NS and TERMCBPER24NS the data doesnt seem to be released every month all the time henc the number of missing values, hence the levels can stay (and be interpolated) however percentage changes related to this variable should be dropped (https://fred.stlouisfed.org/series/TERMCBAUTO48NS 

https://fred.stlouisfed.org/series/TERMCBPER24NS)

In [ ]:
combinedDf.drop([
'fred_TB1YR'
,'fred_TB1YR_LOG'
,'fred_TB1YR_CH1'
,'fred_TB1YR_PC1'
,'fred_TERMCBAUTO48NS_CH1'
,'fred_TERMCBAUTO48NS_LOG'
,'fred_TERMCBAUTO48NS_PC1'
,'fred_TERMCBPER24NS_CH1'
,'fred_TERMCBPER24NS_LOG'
,'fred_TERMCBPER24NS_PC1'], axis=1, inplace=True)


## Perform linear interpolation for missing values:

In [ ]:
missingCols = getMissingCols(combinedDf, prin=True)
print(missingCols)

In [ ]:
combinedDf.interpolate(method='linear',inplace=True)

### Handle remaining missing values

- CCA - Continuously compounded annual rate of change
- CCH - Continuously compounded rate of change
- CHG - Change (from yesterday)
- PCA - Compounded annual rate of change
- PCH - Percentage change (not annually adjusted)

Each of these require one previous value to be defined, hence the initial missing values.
Set each of these initial values to 0.

In [ ]:
changes = ['CCA', 'CCH', 'CHG', 'PCA', 'PCH']
colsToChange = [col for col in missingCols if col[-3:] in changes]

for i in range(len(colsToChange)):
    combinedDf.loc[combinedDf.index[0], colsToChange[i]] = 0
    

In [ ]:
#some columns are uninterpolatable, let's check them out
missingCols = getMissingCols(combinedDf, prin=True)
print(f'{len(missingCols)} columns containing missing values.')

The FRED data missing January 1990 (due to mid-quarter observations) has been interpolated from previous data.

fred_DRTSCILM is quarterly and only began in the second quarter of 1990. There is no way of gathering any previous data on this variable, so in order to avoid nulls this column must be dropped.
The same applies to the NaturalGas variables, since these started in June 1990.

In [ ]:
combinedDf.drop(labels=[colName for colName in combinedDf.columns if 'fred_DRTSCILM' in colName], axis=1, inplace=True)

In [ ]:
printMissingCols(combinedDf)

First 25%-broken Thai rice discontinuity could be linearly interpolated, but 2008 discontinuity is more protracted (5 months), during a period of instability, and the July 2008 price may be unreliable (exactly $700.00 as an average over all rice exports of this kind?) Also already have an indicator on 5%-broken Thai rice, so 25%-broken also likely not useful. Drop.

Cameroonian Sawnwood data too infrequent and lacking granularity - roughly collected once per year every 5 years. Drop.

In [ ]:
combinedDf.drop(labels=['CMO-Historical-Data-Monthly_Rice, Thai 25% ', 'CMO-Historical-Data-Monthly_Sawnwood, Cameroon'], axis=1, inplace=True)

In [ ]:
printMissingCols(combinedDf)

1-month discontinuity for Phosphate rock in November 2023, price then more than halves in December compared to October. Index Mundi (https://www.indexmundi.com/commodities/?commodity=rock-phosphate&months=360) suggests price remained at the same level as October for this month before prices changed, so this will be inserted as an imputation.

In [ ]:
combinedDf.loc['11/2023', 'CMO-Historical-Data-Monthly_Phosphate rock'] = combinedDf.loc['10/2023', 'CMO-Historical-Data-Monthly_Phosphate rock']

In [ ]:
printMissingCols(combinedDf)

## Convert all remaining non-numeric columns to floats

In [ ]:
combinedDf = combinedDf.infer_objects(copy=False)
objectCols = [i for i, el in enumerate(list(combinedDf.dtypes)) if 'float64' not in str(el)]
print(objectCols)
for col in objectCols:
    combinedDf.iloc[:, col] = combinedDf.iloc[:, col].apply(str_to_float)

In [ ]:
#check that there are now no remaining object columns
print([i for i, el in enumerate(list(combinedDf.dtypes)) if 'float64' not in str(el)])

## Train-test split:

In [ ]:
# First remove other PCEPI information to prevent dataleakage:
TargetCol=  combinedDf['fred_'+TARGET].copy()
combinedDf.drop([
'fred_PCEPI',
'fred_PCEPI_CCA',
'fred_PCEPI_CCH',
'fred_PCEPI_CH1',
'fred_PCEPI_CHG',
'fred_PCEPI_PC1',
'fred_PCEPI_PCA',
'fred_PCEPI_PCH',
'fred_PCEPI_LOG'],
axis=1, inplace=True)

X_trainDf= combinedDf.iloc[:combinedDf.index.get_loc(TEST_DATE),:]
X_testDf= combinedDf.iloc[combinedDf.index.get_loc(TEST_DATE):,:]

y_trainDf= TargetCol.iloc[:TargetCol.index.get_loc(TEST_DATE)]
y_testDf= TargetCol.iloc[TargetCol.index.get_loc(TEST_DATE):]

## Granger Causality test:
Granger causality test is used to determine if an exogenous variable causes (granger causes) inflation. This hypothesis test assumes the timeseries are stationary, therefore the timeseries should be proccessed accordingly.

To test for stationarity, Augmented Dickey-Fuller test is used with a significance level of 5%.

The Granger causality test also uses a significance level of 5%

In [ ]:

keepCols=[]# keeps granger caused columns

# Iterate over all columns and test for granger causality
for i in X_trainDf.columns:
    if is_granger_caused(i, y_trainDf, X_trainDf):
        keepCols.append(i)
    
print(len(keepCols))

In [ ]:
caused_train_df= pd.concat([y_trainDf,X_trainDf[keepCols]],axis=1)
caused_test_df= pd.concat([y_testDf,X_testDf[keepCols]],axis=1)

display(caused_train_df)
display(caused_test_df)

## Save data:

In [ ]:
caused_train_df.to_csv(workbookdir.parent / 'Data' /'Train'/f'train{START_YEAR}s.csv')
caused_test_df.to_csv(workbookdir.parent / 'Data' /'Test'/f'test{START_YEAR}s.csv')